# Homework 1

## 1.1

Talk about psychologists and AI for a couple paragraphs.

## 1.2

Describe your TSP problem formulation.

In [5]:
print('Here\'s the problem formulation.')

Here's the problem formulation.


Describe your sample city-domain and how your code does on it.

In [4]:
print('Here\'s your code to run the TSP problem.')

Here's your code to run the TSP problem.


Make some parting comments on this problem.

## 1.3

Follow the same pattern for this problem.

Your solution should represent 6–7 courses (e.g., cs108, cs112, …), 4–5 faculty (e.g., dschuurman, adams, …) course assignments for all courses (e.g., dschuurman teaches 108, adams teaches cs112, …), 4–5 (disjoint!) time slots (mwf900, tth1030, …) and the 2 CS classrooms (e.g., nh253, sb382).

The constraints are that:
- each course should be offered exactly once by the assigned faculty member.
- a faculty member can only teach one thing at a time.
- a room can only have one class at each time.

In [ ]:
def CourseScheduling():
    """A Course scheduling problem"""
    courses = ["cs108", "cs112", "cs214", "cs232", "cs344", "cs262", "cs384"]
    faculty = ["Adams", "Norman", "Schuurman", "Vander Linden"]
    course_assignments = {"cs108": "Norman", "cs112": "Adams", 
                          "cs214": "Adams", "cs232": "Norman", "cs344": "Vander Linden",
                          "cs262": "Vander Linden", "cs384": "Schuurman"}
    time_slots = ["mwf900", "mwf1030", "mwf1130", "tth900", "tth1030"]
    classrooms = ["nh253", "sb382"]
    return CSP(variables, domains, neighbors, constraint)

In [5]:
print('Here\'s yet more code.')

Here's yet more code.
